<a href="https://colab.research.google.com/github/alheliou/Bias_mitigation/blob/main/UPP26/TD2_mepsdata_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Préambule : nos biais inconscients

Nous vous proposons, si vous le souhaitez, de prendre une dizaine de minutes pour tester vos biais inconscients:

https://implicit.harvard.edu/implicit/canadafr/takeatest.html


# TD 2: Manipulation des données

## Objectives


 1. Study the data, the distribution of each feature and its relation to the target.

 2. Highlight some bias present in the data


## Installation of the environnement

We highly recommend you to follow these steps, it will allow every student to work in an environment as similar as possible to the one used during testing.

### Colab Settings ---- for Colab Users ONLY
  The next two cells of code are too execute only once per colab environment


#### 1. Python env creation (Colab only)

        ```
        ! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```

#### 2. Download MEPS dataset (for part2) it can take several minutes (Colab only)

        ```
        ! Rscript /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/generate_data.R
        ! mv h181.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ! mv h192.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ```

  
### Local Settings ---- for installation on local computer ONLY

You can use the same env as TD1, you will then need to follow steps 2 and 4

#### 1. Uv installation (local only, no need to redo if already done)


        https://docs.astral.sh/uv/getting-started/installation/


        `curl -LsSf https://astral.sh/uv/install.sh | sh`

        Python version 3.12 installation (highly recommended)
        `uv python install 3.12`

#### 2. R installation *NEW* (local only)

        In the command `Rscript` says 'command not found'

        `sudo apt install r-base-core`

#### 3. Python env creation (local only, no need to redo if already done)

        ```
        mkdir TD_bias_mitigation
        cd TD_bias_mitigation
        uv python pin 3.12
        uv init
        uv venv
        uv pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```

#### 4. Download MEPS dataset, it can take several minutes *NEW* (local only)

        ```
        cd TD_bias_mitigation/.venv/lib/python3.12/site-packages/aif360/data/raw/meps/
        Rscript generate_data.R
        ```

## Dataset: Meps

We recommend consulting the following pages for a better understanding of the dataset: [MEPSDataset19](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.MEPSDataset19.html) and the [AIF360 tutorial](https://github.com/Trusted-AI/AIF360/blob/main/examples/tutorial_medical_expenditure.ipynb)

What you need to have read
- **The sensitive attribute is 'RACE' :1 is privileged, 0 is unprivileged** ; It is constructed as follows: 'Whites' (privileged class) defined by the features RACEV2X = 1 (White) and HISPANX = 2 (non Hispanic); 'Non-Whites' that included everyone else.
(The features 'RACEV2X', 'HISPANX' etc are removed, and replaced by the 'RACE')
- **'UTILIZATION' is the outcome (the label to predict for a ML model) 0 is positive 1 is negative**. It is a binary composite feature, created to measure the total number of trips requiring some sort of medical care, it sum up the following features (that are removed from the data):
    * OBTOTV15(16), the number of office based visits
    * OPTOTV15(16), the number of outpatient visits
    * ERTOT15(16), the number of ER visits
    * IPNGTD15(16), the number of inpatient nights
    * HHTOTD16, the number of home health visits
UTILISATION is set to 1 when te sum is above or equal to 10, else it is set to 0
- **The dataset is weighted** The dataset come with an 'instance_weights' attribute that corresponds to the feature perwt15f these weights are supposed to generate estimates that are representative of the United State (US) population in 2015.


Summary to remember
- **The sensitive attribute is 'RACE' :1 is privileged, 0 is unprivileged**
- **'UTILIZATION' is the outcome (the label to predict for a ML model) 0 is positive 1 is negative**
- **The dataset is weighted**


In [ ]:
# To execute only in Colab
! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360[inFairness,AdversarialDebiasing] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit

In [ ]:
# Code to compute fairness metrics using aif360

from aif360.sklearn.metrics import *
from sklearn.metrics import  balanced_accuracy_score


# This method takes lists
def get_metrics(
    y_true, # list or np.array of truth values
    y_pred=None,  # list or np.array of predictions
    prot_attr=None, # list or np.array of protected/sensitive attribute values
    priv_group=1, # value taken by the privileged group
    pos_label=1, # value taken by the positive truth/prediction
    sample_weight=None # list or np.array of weights value,
):
    group_metrics = {}
    group_metrics["base_rate_truth"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["base_rate_preds"] = base_rate(
        y_true=y_pred, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        if len(set(y_pred))>1:
            group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
                y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
            )
        else:
            group_metrics["conditional_demographic_disparity"] =None
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["balanced_accuracy_score"] = balanced_accuracy_score(
        y_true=y_true, y_pred=y_pred, sample_weight=sample_weight
        )
    return group_metrics

In this TD we will use data from the [Medical Expenditure Panel Survey](https://meps.ahrq.gov/mepsweb/). The TD is inspired from [AIF360 tutorial](https://github.com/Trusted-AI/AIF360/blob/main/examples/tutorial_medical_expenditure.ipynb)

## The Meps dataset

In [ ]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', append=True, category=UserWarning)


In [ ]:
# Datasets
from aif360.datasets import MEPSDataset19
from aif360.datasets import MEPSDataset20
from aif360.datasets import MEPSDataset21

MEPSDataset19_data = MEPSDataset19()
# (dataset_orig_panel19_train,
#  dataset_orig_panel19_val,
#  dataset_orig_panel19_test) = MEPSDataset19_data.split([0.5, 0.8], shuffle=True)

In [ ]:
(dataset_orig_panel19_train,
 dataset_orig_panel19_val,
 dataset_orig_panel19_test) = MEPSDataset19().split([0.5, 0.8], shuffle=True)

In [ ]:
len(dataset_orig_panel19_train.instance_weights), len(dataset_orig_panel19_val.instance_weights), len(dataset_orig_panel19_test.instance_weights)

In [ ]:
instance_weights = MEPSDataset19_data.instance_weights
instance_weights


In [ ]:
f"Taille du dataset {len(instance_weights)}, poids total du dataset {instance_weights.sum()}."

### Premier appercu du dataset

La librairie AIF360 fournie une surcouche au dataset, cela le rend un peu moins intuitif d'utilisation (par exemple pour étudier/visualiser les attributs un à un), mais elle permet de calculer les métrique des fairness en une ligne de commande.

In [ ]:
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

metric_orig_panel19_train = BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{'RACE': 0}],
        privileged_groups=[{'RACE': 1}])

print(metric_orig_panel19_train.disparate_impact())

Cependant le but de ce TD étant encore de manipuler les données et de les analyser nous allons revenir aux données sous forme d'un dataframe.

Note pour calculer les métriques de fairness sans avoir à les réimplémenter dans le cas pondéré (instances weights) vous pouvez utiliser les méthodes implémenter dans AIF360 là [Implémentation Métriques de Fairness](https://aif360.readthedocs.io/en/latest/modules/sklearn.html#module-aif360.sklearn.metrics)

### Conversion en un dataframe

Nous avons vu que la somme des poids est conséquente, pres de 115millions nous ne pouvons donc pas raisonneblement dupliqué chaque ligne autant de fois que son poids.

Nous allons stocker la pondération et la prendre en compte ensuite dans notre analyse

In [ ]:
def get_df(MepsDataset):
    data = MepsDataset.convert_to_dataframe()
    # data_train est un tuple, avec le data_frame et un dictionnaire avec toutes les infos (poids, attributs sensibles etc)
    df = data[0]
    df['WEIGHT'] = data[1]['instance_weights']
    return df

df = get_df(MEPSDataset19_data)



In [ ]:
from aif360.sklearn.metrics import disparate_impact_ratio, base_rate
dir = disparate_impact_ratio(
    y_true=df.UTILIZATION,
    prot_attr=df.RACE,
    pos_label=0,
    sample_weight=df.WEIGHT)
br =base_rate(
    y_true=df.UTILIZATION,
    pos_label=0,
    sample_weight=df.WEIGHT)
dir,br

In [ ]:
dir = disparate_impact_ratio(
    y_true=df.UTILIZATION,
    prot_attr=df.RACE,
    pos_label=0)
br =base_rate(
    y_true=df.UTILIZATION,
    pos_label=0)
dir,br

## Question 1 - Apprendre un modèle pour prédire le fait d'être réadmis
### 1.1 - Faire le pre-processing des données

Ici ce pre-processing a déjà été fait par AIF, nous avons simplement converti le dataset en dataframe pour pouvoir le manipuler librement

### Question 1.2 - Creer les échantillons d'apprentissage, de validation et de test

Pour créer le df_X il faut enlever l'outcome ("UTILIZATION") et la pondération ("WEIGHT")

La colonne "UTILIZATION" sera le label (noté y)

La colonne "WEIGHT" sera la pondération (notée w)


In [ ]:
# Question 1.2: Créer les échantillons d'apprentissage, de validation et de test

from sklearn.model_selection import train_test_splitprint(f"Test set: {len(X_test)} samples")

print(f"Validation set: {len(X_val)} samples")

# Créer df_X en enlevant UTILIZATION et WEIGHTprint(f"Train set: {len(X_train)} samples")

df_X = df.drop(columns=['UTILIZATION', 'WEIGHT'])

y = df['UTILIZATION'])

w = df['WEIGHT']    X_temp, y_temp, w_temp, test_size=0.375, random_state=42, stratify=y_temp  # 0.375 * 0.8 = 0.3

X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(

# Split en train, validation et test (50%, 30%, 20%)

X_temp, X_test, y_temp, y_test, w_temp, w_test = train_test_split()
    df_X, y, w, test_size=0.2, random_state=42, stratify=y

In [ ]:
df_X.shape, X_train.shape, y_train.shape, w_train.shape, X_val.shape, y_val.shape, w_val.shape,  X_test.shape, y_test.shape, w_test.shape

### Question 1.3 - Apprendre une regression logistique dont le but est de prédire UTILIZATION

In [ ]:
# Question 1.3: Apprendre une régression logistique

from sklearn.linear_model import LogisticRegressionprint(f"Score sur test: {log_reg.score(X_test, y_test, sample_weight=w_test):.4f}")

print(f"Score sur validation: {log_reg.score(X_val, y_val, sample_weight=w_val):.4f}")

# Créer et entraîner le modèle avec pondérationprint(f"Score sur train: {log_reg.score(X_train, y_train, sample_weight=w_train):.4f}")

log_reg = LogisticRegression(max_iter=1000, random_state=42)

log_reg.fit(X_train, y_train, sample_weight=w_train)y_pred_test = log_reg.predict(X_test)

y_pred_val = log_reg.predict(X_val)

# Prédictionsy_pred_train = log_reg.predict(X_train)

### Quesiton 1.4 Performance du modèle (afficher la matrice de confusion)

In [ ]:
# Question 1.4: Matrice de confusion

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplayprint(f"Vrais Positifs (TP): {cm[1, 1]:.0f}")

import matplotlib.pyplot as pltprint(f"Faux Négatifs (FN): {cm[1, 0]:.0f}")

print(f"Faux Positifs (FP): {cm[0, 1]:.0f}")

# Matrice de confusion sur le test setprint(f"\nVrais Négatifs (TN): {cm[0, 0]:.0f}")

cm = confusion_matrix(y_test, y_pred_test, sample_weight=w_test)print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Low Utilization', 'High Utilization'])print("\nMatrice de confusion (pondérée):")

disp.plot(cmap='Blues')

plt.title('Matrice de Confusion - Test Set (Weighted)')plt.show()

### Question 1.5 - Calculer les métriques "group fairness" du modèle

In [ ]:
# Question 1.5: Calculer les métriques "group fairness"

# Extraire l'attribut sensible RACE du test setprint("- Equal Opportunity Difference proche de 0 = équitable")

race_test = X_test['RACE'].valuesprint("- Statistical Parity Difference proche de 0 = équitable")

print("- Disparate Impact Ratio proche de 1 = équitable")

# Calculer les métriques de fairness avec pondérationprint("\nInterprétation:")

print("=== Métriques de Group Fairness (RACE) ===")

print("Groupe privilégié: White (RACE=1) | Groupe défavorisé: Non-White (RACE=0)\n")        print(f"{metric_name}: None")

    else:

metrics = get_metrics(        print(f"{metric_name}: {metric_value:.4f}")

    y_true=y_test.values,    if metric_value is not None:

    y_pred=y_pred_test,for metric_name, metric_value in metrics.items():

    prot_attr=race_test,

    priv_group=1,)

    pos_label=0,  # 0 est le label positif selon la description    sample_weight=w_test.values

## Question 2 - Etude de l'impact de la couleur de peau sur le faire d'être réadmis, les prédictions du modèle et ses liens avec les autres variables


### Question 2.1 - Faire l'étude descriptive univarié de la couleur de peau ('RACE') (effectif, fréquence, model)

In [ ]:
# Question 2.1: Étude descriptive univariée de RACE

print("=== Analyse Univariée de RACE ===")print("Mode (pondéré):", race_weighted.idxmax())

print("\nEffectifs:")print("\nMode (non pondéré):", df['RACE'].mode().values[0])

print(df['RACE'].value_counts().sort_index())

print(race_weighted / race_weighted.sum())

print("\nEffectifs pondérés:")print("\nFréquences pondérées:")

race_weighted = df.groupby('RACE')['WEIGHT'].sum()

print(race_weighted)print(df['RACE'].value_counts(normalize=True).sort_index())

print("\nFréquences:")

### Question 2.2 - Faire des graphiques décrivant la couleur de peau  (diagramme en secteur, diagramme en barres)

In [ ]:
# Question 2.2: Graphiques décrivant RACE

import plotly.graph_objects as gofig.show()

             title='Distribution de RACE - Diagramme en barres (non pondéré)')

# Diagramme en secteur (pondéré)             labels={'x': 'RACE', 'y': 'Nombre d\'observations'},

race_counts = df.groupby('RACE')['WEIGHT'].sum()fig = px.bar(x=labels, y=race_counts_unweighted.values,

labels = ['Non-White (0)', 'White (1)']race_counts_unweighted = df['RACE'].value_counts().sort_index()

fig = px.pie(values=race_counts.values, names=labels, # Diagramme en barres (non pondéré)

             title='Distribution de RACE (pondérée)')

fig.show()fig.show()

             title='Distribution de RACE - Diagramme en barres (pondéré)')

# Diagramme en barres (pondéré)             labels={'x': 'RACE', 'y': 'Poids total'},
fig = px.bar(x=labels, y=race_counts.values,

### Question 2.3 - Faire l'analyse bivariée entre la couleur de peau et les autres variables explicatives quantitatives (boite à moustaches des variables par genre, densité/histogramme par genre, rapport de corrélation)

In [ ]:
# Question 2.3: Analyse bivariée RACE vs variables quantitatives

# Variables quantitatives dans le dataset    print(f"{var}: {eta_squared:.4f}")

quant_vars = df.select_dtypes(include=[np.number]).columns.tolist()    eta_squared = ss_between / ss_total if ss_total > 0 else 0

quant_vars = [v for v in quant_vars if v not in ['RACE', 'UTILIZATION', 'WEIGHT']]    ss_total = sum([(x - grand_mean)**2 for x in df[var]])

    ss_between = sum([len(g) * (np.mean(g) - grand_mean)**2 for g in groups])

print(f"Variables quantitatives analysées: {quant_vars[:5]}...")  # Afficher quelques-unes    grand_mean = df[var].mean()

    groups = df.groupby('RACE')[var].apply(list)

# Boxplots pour quelques variables clés    # Calculer eta²

for var in quant_vars[:3]:  # Afficher les 3 premières pour exemplefor var in quant_vars[:5]:

    fig = px.box(df, x='RACE', y=var, print("\nRapport de corrélation (eta²) entre RACE et variables quantitatives:")

                 title=f'Boxplot de {var} par RACE',# Rapport de corrélation (eta²) pour mesurer la dépendance

                 labels={'RACE': 'RACE (0=Non-White, 1=White)'})

    fig.show()    fig.show()

                       barmode='overlay', opacity=0.7)

# Histogrammes par RACE pour une variable                       title=f'Distribution de {var} par RACE',

if len(quant_vars) > 0:    fig = px.histogram(df, x=var, color='RACE',
    var = quant_vars[0]

### Question 2.4 - Faire l'analyse bivariée entre la couelur de peau et d'autres variables explicatives qualitative (table de contingence, diagramme en barre selon les profils lignes et selon les profils colonnes, diagramme en mosaique)

In [ ]:
# Question 2.4: Analyse bivariée RACE vs variables qualitatives

# Variables qualitatives (on peut considérer certaines variables binaires comme qualitatives)print(f"\nTest du Chi²: chi2={chi2:.2f}, p-value={p_value:.4f}")

# Pour cet exemple, créons une variable catégorielle à partir de l'âge si disponiblechi2, p_value, dof, expected = chi2_contingency(contingency_weighted)

from scipy.stats import chi2_contingency

# Table de contingence avec UTILIZATION# Test du Chi²

print("=== Table de contingence: RACE x UTILIZATION ===")

contingency = pd.crosstab(df['RACE'], df['UTILIZATION'])fig.show()

print(contingency)             barmode='group')

             labels={'value': 'Pourcentage', 'RACE': 'RACE (0=Non-White, 1=White)'},

print("\nTable de contingence pondérée:")             title='Profils lignes: Distribution de UTILIZATION par RACE',

contingency_weighted = pd.crosstab(df['RACE'], df['UTILIZATION'],              x='RACE', y='value', color='UTILIZATION',

                                    values=df['WEIGHT'], aggfunc='sum')fig = px.bar(profiles_row.reset_index().melt(id_vars='RACE'),

print(contingency_weighted)# Diagramme en barres groupées



# Profils lignes (distribution de UTILIZATION par RACE)print(profiles_col)

print("\nProfils lignes (% par ligne):")profiles_col = contingency_weighted.div(contingency_weighted.sum(axis=0), axis=1) * 100

profiles_row = contingency_weighted.div(contingency_weighted.sum(axis=1), axis=0) * 100print("\nProfils colonnes (% par colonne):")

print(profiles_row)# Profils colonnes (distribution de RACE par UTILIZATION)


### Question 2.5 - Faire l'analyse bivariée entre la couleur de peau et la colonne 'UTILIZATION'

In [ ]:
# Question 2.5: Analyse bivariée RACE vs UTILIZATION

print("=== Analyse détaillée: RACE vs UTILIZATION ===")print(f"\nDisparate Impact Ratio (UTILIZATION): {dir_util:.4f}")

)

# Taux d'utilisation par race (pondéré)    sample_weight=df['WEIGHT']

util_by_race = df.groupby('RACE').apply(    pos_label=0,

    lambda x: (x['UTILIZATION'] == 0).sum() * x['WEIGHT'].iloc[0] if len(x) > 0 else 0    priv_group=1,

)    prot_attr=df['RACE'],

total_by_race = df.groupby('RACE')['WEIGHT'].sum()    y_true=df['UTILIZATION'],

rate_by_race = util_by_race / total_by_racedir_util = disparate_impact_ratio(

from aif360.sklearn.metrics import disparate_impact_ratio

print("\nTaux d'utilisation élevée (UTILIZATION=0) par RACE:")# Calculer le disparate impact

for race in [0, 1]:

    print(f"RACE {race}: {rate_by_race[race]:.4f}")fig.show()

                   y='WEIGHT')

# Graphique                   histfunc='sum',

fig = px.histogram(df, x='RACE', color='UTILIZATION',                   barmode='group',

                   title='Distribution de UTILIZATION par RACE',                   labels={'RACE': 'RACE (0=Non-White, 1=White)'},

                   labels={'RACE': 'RACE (0=Non-White, 1=White)'},                   title='Distribution de UTILIZATION par RACE (avec pondération)',

                   barmode='group')fig = px.histogram(df_plot, x='RACE', color='UTILIZATION',

fig.show()df_plot = df.copy()

# Avec pondération

### Question 2.6 - Faire l'analyse bivariée entre la couleur de peau et les prédictions du modèle prédisant la colonne 'UTILIZATION'

In [ ]:
# Question 2.6: Analyse bivariée RACE vs prédictions du modèle

print("=== Analyse: RACE vs Prédictions du Modèle ===")print(f"\nDisparate Impact Ratio (Prédictions): {dir_pred:.4f}")

)

# Prédictions sur l'ensemble du dataset    sample_weight=df_with_pred['WEIGHT']

y_pred_all = log_reg.predict(df_X)    pos_label=0,

    priv_group=1,

# Taux de prédictions positives par race    prot_attr=df_with_pred['RACE'],

df_with_pred = df.copy()    y_true=df_with_pred['PREDICTION'],

df_with_pred['PREDICTION'] = y_pred_alldir_pred = disparate_impact_ratio(

# Disparate impact sur les prédictions

# Calculer les taux par race (pondéré)

for race in [0, 1]:print(contingency_pred)

    mask = df_with_pred['RACE'] == raceprint("\nTable de contingence pondérée (RACE x PREDICTION):")

    rate_pos = ((df_with_pred[mask]['PREDICTION'] == 0) * df_with_pred[mask]['WEIGHT']).sum() / df_with_pred[mask]['WEIGHT'].sum()                                values=df_with_pred['WEIGHT'], aggfunc='sum')

    print(f"RACE {race}: Taux de prédiction positive (PREDICTION=0): {rate_pos:.4f}")contingency_pred = pd.crosstab(df_with_pred['RACE'], df_with_pred['PREDICTION'],

# Table de contingence

# Graphique

fig = px.histogram(df_with_pred, x='RACE', color='PREDICTION',fig.show()

                   title='Distribution des Prédictions par RACE',                   barmode='group')
                   labels={'RACE': 'RACE (0=Non-White, 1=White)'},

### Question 2.7 - Faire l'analyse bivariée entre la couleur de peau et les erreurs du modèle précédent

In [ ]:
# Question 2.7: Analyse bivariée RACE vs erreurs du modèle

print("=== Analyse: RACE vs Erreurs du Modèle ===")fig.show()

                   barmode='stack')

# Créer une colonne d'erreurs                   labels={'RACE': 'RACE (0=Non-White, 1=White)'},

df_with_pred['ERROR'] = (df_with_pred['UTILIZATION'] != df_with_pred['PREDICTION']).astype(int)                   title='Types d\'Erreurs par RACE',

fig = px.histogram(df_with_pred, x='RACE', color='ERROR_TYPE',

# Taux d'erreur par race

for race in [0, 1]:df_with_pred.loc[(df_with_pred['UTILIZATION'] == 0) & (df_with_pred['PREDICTION'] == 1), 'ERROR_TYPE'] = 'Faux Négatif'

    mask = df_with_pred['RACE'] == racedf_with_pred.loc[(df_with_pred['UTILIZATION'] == 1) & (df_with_pred['PREDICTION'] == 0), 'ERROR_TYPE'] = 'Faux Positif'

    error_rate = (df_with_pred[mask]['ERROR'] * df_with_pred[mask]['WEIGHT']).sum() / df_with_pred[mask]['WEIGHT'].sum()df_with_pred['ERROR_TYPE'] = 'Correct'

    print(f"RACE {race}: Taux d'erreur: {error_rate:.4f}")# Analyse des types d'erreurs



# Types d'erreurs par racefig.show()

for race in [0, 1]:                   barmode='group')

    mask = df_with_pred['RACE'] == race                           'ERROR': 'Erreur (0=Correct, 1=Erreur)'},

    df_race = df_with_pred[mask]                   labels={'RACE': 'RACE (0=Non-White, 1=White)', 

                       title='Distribution des Erreurs par RACE',

    # Faux positifsfig = px.histogram(df_with_pred, x='RACE', color='ERROR',

    fp = ((df_race['UTILIZATION'] == 1) & (df_race['PREDICTION'] == 0)).sum()# Visualisation des erreurs par race

    # Faux négatifs

    fn = ((df_race['UTILIZATION'] == 0) & (df_race['PREDICTION'] == 1)).sum()    print(f"  Faux Négatifs: {fn}")

        print(f"  Faux Positifs: {fp}")
    print(f"\nRACE {race}:")

### Question 2.8 - Proposer un modèle à base d'une foret aléatoire prédisant la couleur de peau en fonction des autres variables explicatives

In [ ]:
# Question 2.8: Modèle de forêt aléatoire prédisant RACE

from sklearn.ensemble import RandomForestClassifierprint("à partir des autres variables, suggérant un risque de proxy discrimination.")

from sklearn.metrics import classification_report, roc_auc_scoreprint("\nInterprétation: Un score élevé indique que RACE est fortement prédictible")



print("=== Modèle prédisant RACE à partir des autres variables ===")fig.show()

             title='Importance des Features pour Prédire RACE')

# Préparer les données (enlever RACE, UTILIZATION, WEIGHT)             orientation='h',

X_race = df.drop(columns=['RACE', 'UTILIZATION', 'WEIGHT'])fig = px.bar(feature_importance.head(10), x='importance', y='feature',

y_race = df['RACE']# Visualisation

w_race = df['WEIGHT']

print(feature_importance.head(10))

# Split train/testprint("\nTop 10 features les plus importantes:")

X_train_race, X_test_race, y_train_race, y_test_race, w_train_race, w_test_race = train_test_split(

    X_race, y_race, w_race, test_size=0.2, random_state=42, stratify=y_race}).sort_values('importance', ascending=False)

)    'importance': rf_race.feature_importances_

    'feature': X_race.columns,

# Entraîner une forêt aléatoirefeature_importance = pd.DataFrame({

rf_race = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)# Importance des features

rf_race.fit(X_train_race, y_train_race, sample_weight=w_train_race)

print(f"AUC-ROC: {roc_auc_score(y_test_race, y_proba_race, sample_weight=w_test_race):.4f}")

# Prédictionsprint(f"\nScore sur le test set: {rf_race.score(X_test_race, y_test_race, sample_weight=w_test_race):.4f}")

y_pred_race = rf_race.predict(X_test_race)# Performance

y_proba_race = rf_race.predict_proba(X_test_race)[:, 1]

### Question 2.9 - Apprendre un modèle privé de la couleur de peau pour prédire UTILIZATION et étudier le lien entre ses prédictions et la couleur de peau

In [ ]:
# Question 2.9: Modèle sans RACE pour prédire UTILIZATION

print("=== Modèle sans RACE pour prédire UTILIZATION ===")print(f"Score sans RACE: {log_reg_no_race.score(X_test_nr, y_test_nr, sample_weight=w_test_nr):.4f}")

print(f"Score avec RACE: {log_reg.score(X_test, y_test, sample_weight=w_test):.4f}")

# Créer les données sans RACEprint("\n=== Comparaison des modèles ===")

X_no_race = df.drop(columns=['RACE', 'UTILIZATION', 'WEIGHT'])# Comparaison avec le modèle incluant RACE

y_util = df['UTILIZATION']

w_util = df['WEIGHT']print("cela suggère une discrimination par proxy via d'autres variables corrélées à RACE.")

print("\nInterprétation: Si le disparate impact persiste même sans RACE,")

# Split train/test

X_train_nr, X_test_nr, y_train_nr, y_test_nr, w_train_nr, w_test_nr = train_test_split(    print(f"RACE {race}: Taux de prédiction positive: {rate:.4f}")

    X_no_race, y_util, w_util, test_size=0.2, random_state=42, stratify=y_util    rate = ((y_pred_nr[mask] == 0) * w_test_nr.values[mask]).sum() / w_test_nr.values[mask].sum()

)    mask = race_test_nr == race

for race in [0, 1]:

# Entraîner une régression logistique sans RACE# Taux de prédictions positives par race

log_reg_no_race = LogisticRegression(max_iter=1000, random_state=42)

log_reg_no_race.fit(X_train_nr, y_train_nr, sample_weight=w_train_nr)print(f"Disparate Impact Ratio des prédictions: {dir_no_race:.4f}")

)

# Prédictions    sample_weight=w_test_nr.values

y_pred_nr = log_reg_no_race.predict(X_test_nr)    pos_label=0,

    priv_group=1,

print(f"\nScore sur le test set: {log_reg_no_race.score(X_test_nr, y_test_nr, sample_weight=w_test_nr):.4f}")    prot_attr=race_test_nr,

    y_true=y_pred_nr,

# Analyser le lien entre les prédictions et RACEdir_no_race = disparate_impact_ratio(

race_test_nr = df.loc[y_test_nr.index, 'RACE'].values# Disparate impact


print("\n=== Lien entre les prédictions (sans RACE) et RACE ===")

## Question 3 - Faire de meme pour une autre variable sensible (Age, genre, marry etc)


In [ ]:
# Question 3: Analyse pour une autre variable sensible

print("=== Analyse pour une autre variable sensible: Exemple avec une variable d'âge ===")    print("Vous pouvez créer une variable d'âge binaire (jeune/âgé) ou autre variable catégorielle.")

    print("\nPas assez de variables binaires pour l'analyse.")

# Explorer les colonnes disponibleselse:

print("\nColonnes disponibles dans le dataset:")                print(f"{metric_name}: {metric_value:.4f}")

print(df.columns.tolist())            if metric_value is not None:

        for metric_name, metric_value in metrics_alt.items():

# Créer une variable binaire d'âge si elle n'existe pas déjà        print(f"\n=== Métriques de Fairness pour {sensitive_var} ===")

# (adapter selon les colonnes réellement présentes dans le dataset)        

        )

# Exemple d'analyse pour toute variable binaire disponible            sample_weight=w_test.values

binary_cols = [col for col in df.columns if df[col].nunique() == 2 and col not in ['UTILIZATION', 'WEIGHT']]            pos_label=0,

            priv_group=priv_value,

if len(binary_cols) > 1:            prot_attr=sensitive_test,

    sensitive_var = binary_cols[0]  # Prendre la première variable binaire autre que RACE            y_pred=y_pred_test,

    print(f"\nAnalyse de la variable sensible: {sensitive_var}")            y_true=y_test.values,

            metrics_alt = get_metrics(

    # Distribution        

    print(f"\nDistribution de {sensitive_var}:")        priv_value = df[sensitive_var].mode().values[0]

    print(df[sensitive_var].value_counts())        sensitive_test = X_test[sensitive_var].values

        if sensitive_var in X_test.columns:
    # Métriques de fairness

## Question 4 - Causalité: appliquer le module causal-learn sur le jeu de données en testant plusieurs des méthodes implémentées et en comparant les résultats entre les méthodes

In [ ]:
# Question 4: Causalité avec causal-learn

print("=== Analyse de Causalité avec causal-learn ===")print("Des chemins indirects suggèrent une médiation par d'autres variables.")

print("Une arête de RACE vers UTILIZATION suggère une influence causale directe.")

from causallearn.search.ConstraintBased.PC import pcprint("Comparez les résultats des différentes méthodes pour identifier les relations robustes.")

from causallearn.utils.GraphUtils import GraphUtilsprint("Les graphes causaux montrent les relations de causalité potentielles entre variables.")

import matplotlib.image as mpimgprint("\n=== Interprétation ===")

import matplotlib.pyplot as plt

    print(f"Erreur avec GES: {e}")

# Préparer un sous-ensemble de données pour l'analyse causaleexcept Exception as e:

# (prendre un échantillon car les algorithmes peuvent être lents)    plt.show()

np.random.seed(42)    plt.title('Graphe Causal - Algorithme GES')

sample_size = min(1000, len(df))    plt.axis('off')

sample_indices = np.random.choice(len(df), sample_size, replace=False)    plt.imshow(img)

    plt.figure(figsize=(12, 8))

# Sélectionner quelques variables clés    img = mpimg.imread('causal_graph_ges.png')

key_vars = ['RACE', 'UTILIZATION'] + [col for col in df.columns if col not in ['RACE', 'UTILIZATION', 'WEIGHT']][:5]    

df_causal = df.iloc[sample_indices][key_vars].copy()    pyd.write_png('causal_graph_ges.png')

    pyd = GraphUtils.to_pydot(record['G'], labels=key_vars)

print(f"\nVariables utilisées pour l'analyse causale: {key_vars}")    

print(f"Taille de l'échantillon: {len(df_causal)}")    print("Graphe causal obtenu avec GES algorithm")

    record = ges(data_array, score_func='local_score_BIC')

# Convertir en array numpy    

data_array = df_causal.values    from causallearn.search.ScoreBased.GES import ges

try:

# Méthode 1: PC Algorithm (Peter-Clark)print("\n--- Algorithme GES (Score-Based) ---")

print("\n--- Algorithme PC (Constraint-Based) ---")# Méthode 2: GES (Greedy Equivalence Search)

try:

    cg_pc = pc(data_array, alpha=0.05, indep_test='fisherz')    print(f"Erreur avec PC: {e}")

    print("Graphe causal obtenu avec PC algorithm")except Exception as e:

        print(cg_pc.G.graph)

    # Visualiser le graphe    print("\nArêtes du graphe:")

    pyd = GraphUtils.to_pydot(cg_pc.G, labels=key_vars)    

    pyd.write_png('causal_graph_pc.png')    plt.show()

        plt.title('Graphe Causal - Algorithme PC')

    img = mpimg.imread('causal_graph_pc.png')    plt.axis('off')

    plt.figure(figsize=(12, 8))    plt.imshow(img)